# MATH 210/215 Project C

Doreen Cai: 81597270

Ethan Chen: 64423676

Liu Sun: 72619521

Yeyang\(David\) Ou: 93180594 



### Spruce Budworm Infestation of Balsam Fir Trees

In this project, we will analyze a set of ordinary differential equations \(ODEs\) that represent a model for analyzing the impact of spruce budworms on balsam fir forests. Fundamentally, the model outlines how a host population \(the trees\) is gradually affected by a parasitic population \(the budworms\).  As the trees progressively lose biomass to these parasites, the budworms' sustainable population level correspondingly diminishes.  When faced with certain predatory behaviors, such as birds preying on the budworms, there is a sharp decline in the budworm population as it reaches a critical level of support.  Consequently, this allows the tree population to slowly rebuild past the point where budworm numbers plummet, enabling their numbers to grow again. The dynamics repeat themselves.  
$$ 
\frac{dB}{dt}= B(1-\frac{B}{\alpha S})-\beta \frac{B^n}{S^n+B^n}
$$ 
$$
\frac{dS}{dt} =\epsilon(S(1-S)-\delta B)
$$

Variables are described as:

- $B$: budworm population density
- $S$: total surface area of tree branches



#### Parameter Values

We will define our parameters as followed:

- Sensitivity of budworm carrying capacity to branch loss, alpha $\alpha = 8$

- Insensitivity of predation by birds, beta $\beta = 1.5$

- Ratio of growth time scales \(tree:budworm\), epsilon $\epsilon = 0.05$ 

- Per capita impact of budworm on trees, delta $\delta = 0.1$ 

- Power of predation, $n = 2$ 



In [371]:
import numpy as np
import matplotlib.pyplot as plt 
import scipy.integrate as spi

## Part 1: Plotting the $\frac{dB}{dt}$ and $\frac{dS}{dt}$ Function



For the first part, we will first plot the  $\frac{dB}{dt}$ and $\frac{dS}{dt}$ function.



In [372]:
# Define vectors

alpha = 8
beta = 1.5
n = 2
epsilon = 0.05
delta = 0.1

In [373]:
# Define functions and Run to see the plot
f = lambda B,S: B*(alpha*S-B)*(S**n+B**n)-alpha*beta*S*B**n
g = lambda B,S: epsilon*(S*(1-S)-delta*B)

B = np.linspace(-0.5,3,100)
S = np.linspace(-1,2,100)
X,Y = np.meshgrid(B,S)
Z = f(X,Y)
Q = g(X,Y)


plt.contour(B,S,Z,levels=0,colors='r')
plt.contour(B,S,Q,levels=0,colors='b')
plt.xlabel('B (budworm population density)')
plt.ylabel('S (total surface area of tree branches)')
plt.grid(True)
plt.show()

In [374]:
# Minimize the coordinates to find the steay states
B = np.linspace(1,2,100)
S = np.linspace(0.7,0.9,100)
X,Y = np.meshgrid(B,S)
Z = f(X,Y)
Q = g(X,Y)


plt.contour(B,S,Z,levels=0,colors='r')
plt.contour(B,S,Q,levels=0,colors='b')
plt.xlabel('B (budworm population density)')
plt.ylabel('S (total surface area of tree branches)')
plt.grid(True)
plt.show()


## Part 2: Finding The Steady State



The steady states occur at the intersection of the nullclines \(when $f(x,y) = g(x,y) = 0$ \), we will use the `scipy.optimize` function to solve for the steady state. By looking at the graph closely, there are 5 intersections. The two graphs intersect with each other three times between $B = 1$ and $B = 2$ and there are two steady states at $B= 0$. However, we will be ignoring the 2 intersection at $B= 0$ since $ \frac{dB}{dt}= B(1-\frac{B}{\alpha S})-\beta \frac{B^n}{S^n+B^n}$ will be zero \(hence no population\). So, we will only consider 3 steady states.


In [375]:
import scipy.optimize as spo

#Define the function and guess the steady states points
F = lambda v: np.array([f(*v),g(*v)])
v0 = [1,0.9]
v1 = [1.6,0.8]
v2 = [1.8,0.8]

ss0 = spo.fsolve(F,v0)
ss1 = spo.fsolve(F,v1)
ss2 = spo.fsolve(F,v2)

print('Steady State 1:', ss0)
print('Steady State 2:', ss1)
print('Steady State 3:', ss2)


Steady State 1: [1.20514474 0.85984097]
Steady State 2: [1.6 0.8]
Steady State 3: [1.78461676 0.76746649]


We found that there are 3 steady states:

* **Steady State 1:** \[1.20514474 0.85984097\]
* **Steady State 2:** \[1.6 0.8\]
* **Steady State 3:** \[1.78461676 0.76746649\]



## Part 3: Approximating the Solutions of the System of ODEs



To use `scipy.integrate.odeint` to approximate solutions of our system of ODEs, we have to write our functions as a first order system of equations.

Let $u_0 = B, u_1 = B', u_2 = S, u_3 = S'$

- $u_0' = u_1$
- $u_1' = B(1-\frac{B}{\alpha S})-\beta \frac{B^n}{S^n+B^n}$
- $u_2' = u_3$
- $u_3' = \epsilon(S(1-S)-\delta B)$



In [376]:
# Define vectors and plot B and S
B = np.linspace(0,5,100)
S = np.linspace(0,1.2,100)
X,Y = np.meshgrid(B,S)
Z = f(X,Y)
Q = g(X,Y)
plt.contour(B,S,Z,levels=0,colors='r')
plt.contour(B,S,Q,levels=0,colors='b')

# Define ODEs function
def h(u, t):
    B, S = u
    dBdt = B*(1-B/(alpha*S)) - beta*(B**n)/(S**n+B**n)
    dSdt = epsilon * (S * (1 - S) - delta * B)
    return [dBdt, dSdt]

# Set initial conditions
initial_values = [[0.1,0.1],[0.2,0.2],[0.3,0.3],[0.5,0.5],[0.7,0.7],[1.0,1.0]]

# Time points to evaluate the ODEs
t = np.linspace(0, 100, 1000)


# Solve the system for each initial value
for B0, S0 in initial_values:
    
    # Integrate the ODEs for each set of initial conditions
    sol = spi.odeint(h, [B0,S0], t)  

    # Plot trajectory
    plt.plot(sol[:, 0], sol[:, 1], label=f'B(0)={B0}, S(0)={S0}')

# Add labels and legend
plt.title('Trajectories of the system of ODEs')
plt.xlabel('Budworm Population Density (B)')
plt.ylabel('Total Surface Area of Tree Branches (S)')
plt.legend()

# Show plots
plt.grid(True)
plt.show()

From the graphs above, we can see the trajectories of the system of ODEs plotted for different initial conditions of B\(0\) and S\(0\). Each trajectory represents how the budworm population density \(B\) and the total surface area of tree branches \(S\) evolve over time. 



### Part 4: Change in Behaviors Varying Beta \($\beta$\)  and Delta\($\delta$ \)



We choose to vary parameters $\beta$ and $\delta$. Let's vary $\beta$ between 1.2 and 1.8 \(which is ±20% of 1.5\) and $\delta$ between 0.08 and 0.12 \(which is ±20% of 0.1\). 


In [377]:
# Define a range for beta and delta parameters
beta_values = np.linspace(1.2, 1.8, 5)  # Vary β between 1.2 and 1.8
delta_values = np.linspace(0.08, 0.12, 5)  # Vary δ between 0.08 and 0.12

# Initial conditions and time points
initial_conditions = ([0.5,0.5])
t = np.linspace(0, 100, 1000)

# Simulate and plot the trajectories
for beta in beta_values:
    for delta in delta_values:
        # Integrate the system of ODEs
        trajectory = spi.odeint(h, initial_conditions, t)
        plt.plot(trajectory[:, 0], trajectory[:, 1], label=f'β={beta:.2f}, δ={delta:.2f}')

# Labeling the axes and show the plot
plt.xlabel('Budworm Population Density (B)')
plt.ylabel('Total Surface Area of Tree Branches (S)')
plt.title('Trajectories for Varied β and δ')
plt.legend()
plt.show()

From the plot, we can see varying $\beta$ and $\delta$ leads to both quantitative changes\(such as the extent of budworm population and tree surface area\)and qualitative changes\(such as the existence, stability\).
